## Model Comparison Jupyter notebook

This notebook will load nets as defined int the imported python modules with certain preformance characteristics for comparison.
This file will also contain the tensorboard that helps visualize the models accuracy over time and the training process.

Components:
- initialize training parameters and fetch dataset (including RotMNIST dataset)
- compare model attributes (such as total parameters and structure)
- define hyperparameters
- create tensorboard and set up preformance graphs
- compare model training and preformance under different conditions
- save models to files

In [1]:
### Imports for pytorch and dataset

import torch
from torch.autograd import Variable

import torchvision
from torchvision import datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

import torch.nn as nn

from torchvision.datasets.cifar import CIFAR10

In [2]:
### Defining simple composition for rotations for following compose
import torchvision.transforms.functional as TF
import random

class RotationP4:
    """Rotate randomly in p4 group"""

    def __init__(self):
        self.angles = [-90, 0, 90, 180]
        
    def __call__(self, x):
        angle = random.choice(self.angles)
        return TF.rotate(x, angle)

augmented_transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    RotationP4(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

normal_transforms = transform_train = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
### Define dataloaders
### Instatiate RotMNIST and verify behaviour below with the dataloaders
dataset_rot = CIFAR10(
    root = 'data',
    download=True,
    train=True,
    transform=augmented_transform
)

test_dataset_rot = CIFAR10(
    root = 'data',
    download=True,
    train=False,
    transform=augmented_transform
)

dataset_upright = CIFAR10(
    root = 'data',
    download=True,
    train=True,
    transform=normal_transforms,
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
### Import different networks from python files
# TODO: Uncomment other networks and import
from p4m_conv import P4MNetC
from p4_conv import P4NetC
from z2_conv import ConvNetC

p4m_net = P4MNetC()
p4_net = P4NetC()
conv_net = ConvNetC()

p4m_total_params = sum(p.numel() for p in p4m_net.parameters() if p.requires_grad)
p4_total_params = sum(p.numel() for p in p4_net.parameters() if p.requires_grad)
z2_total_params = sum(p.numel() for p in conv_net.parameters() if p.requires_grad)

print(p4m_net)
print(p4_net)
print(conv_net)

print("P4M  --\tTrainable Params: " + str(p4m_total_params))
print("P4   --\tTrainable Params: " + str(p4_total_params))
print("Conv --\tTrainable Params: " + str(z2_total_params))

P4MNetC(
  (conv1): P4MConvZ2()
  (conv2): P4MConvP4M()
  (conv3): P4MConvP4M()
  (fc1): Linear(in_features=48, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)
P4NetC(
  (conv1): P4ConvZ2()
  (conv2): P4ConvP4()
  (conv3): P4ConvP4()
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)
ConvNetC(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(24, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)
P4M  --	Trainable Params: 290882
P4   --	Trainable Params: 259722
Conv --	Tra

In [5]:
### Hyperparameters
learning_rate = 0.001
batch_size = 64
epochs = 128

### Objectives/Loss fn
loss_fn = nn.CrossEntropyLoss()

### Dataloaders
train_dataloader_rot = DataLoader(dataset_rot, batch_size=batch_size, shuffle=True)
test_dataloader_rot = DataLoader(test_dataset_rot, batch_size=batch_size, shuffle=True)
train_dataloader_upright = DataLoader(dataset_upright, batch_size=batch_size, shuffle=True)

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer, cur_epoch):
    running_loss = 0.0
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        model.to(device)
        # Compute prediction and loss for backprop
        pred = model(X.to(device))
        loss = loss_fn(pred, y.to(device))

        # Backpropagation by setting grad to zero, calculating using backprop engine and stepping (using learning rate)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if batch % 100 == 99:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

            writer.add_scalar('training loss', running_loss / 100, cur_epoch * len(dataloader) + batch)
            running_loss = 0.0

def test_loop(dataloader, model, loss_fn):
    model.to(device)
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0
    
    # No gradient on training data (faster computation and no optimization happening here anyway)
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.to(device))
            test_loss += loss_fn(pred, y.to(device)).item()
            correct += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct

In [7]:
### Train all networks
def train_test_net(net, train_upright):
    # Add option to train networks with RotMNIST
    test_dataloader = train_dataloader_rot
    if (train_upright):
        test_dataloader = train_dataloader_upright

    optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loop(test_dataloader, net, loss_fn, optimizer, t)
        correct = test_loop(test_dataloader_rot, net, loss_fn)
        writer.add_scalar('Test Performance', correct, t * len(test_dataloader_rot) + batch_size)
    print('Finished Training Net + ' + str(type(net)))

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import torch.utils.tensorboard
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/p4m_CIFAR_1')
train_test_net(p4m_net, True)

writer = SummaryWriter('runs/p4_CIFAR_1')
train_test_net(p4_net, True)

writer = SummaryWriter('runs/conv_CIFAR_1')
train_test_net(conv_net, False)

convu_net = ConvNetC()
writer = SummaryWriter('runs/convu_CIFAR_1')
train_test_net(convu_net, True)

Epoch 1
-------------------------------
loss: 2.294169  [ 6336/50000]
loss: 2.114599  [12736/50000]
loss: 1.902977  [19136/50000]
loss: 2.069120  [25536/50000]
loss: 1.905852  [31936/50000]
loss: 1.983189  [38336/50000]
loss: 1.844396  [44736/50000]
Test Error: 
 Accuracy: 34.8%, Avg loss: 0.028136 

Epoch 2
-------------------------------
loss: 1.862592  [ 6336/50000]
loss: 1.876931  [12736/50000]
loss: 1.754698  [19136/50000]
loss: 1.625921  [25536/50000]
loss: 1.561831  [31936/50000]
loss: 1.564086  [38336/50000]
loss: 1.788957  [44736/50000]


### Model Comparisons

There are a few comparisons between the models to be made here. Here is a list of the following that I log
- Model accuracy on 10000 test images
- Model accuracy per class

In [ ]:
def test_accuracy(net):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in test_dataloader_rot:
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.to(device)).sum().item()

    print('Accuracy of the ' + str(type(net)) + ' on the 10000 test images: %f %%' % (
        100.0 * correct / total))

In [ ]:
def class_labels(net):
    
    classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    # prepare to count predictions for each class
    correct_pred = {num : 0 for num in range(0, 10)}
    total_pred = {num : 0 for num in range(0, 10)}

    # again no gradients needed
    with torch.no_grad():
        for data in test_dataloader_rot:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images.to(device))
            _, predictions = torch.max(outputs, 1)
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1

    print('Classes for ' + str(type(net)))

    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print("Accuracy for num {} is: {:.1f} %".format(classname,
                                                   accuracy))

In [ ]:
test_accuracy(p4m_net)
class_labels(p4m_net)

test_accuracy(p4_net)
class_labels(p4_net)

test_accuracy(conv_net)
class_labels(conv_net)

test_accuracy(convu_net)
class_labels(convu_net)

In [ ]:
## TODO: Uncomment
torch.save(p4m_net, 'upright-trained-p4m-cifar-1.pth')
torch.save(p4_net, 'upright-trained-p4-cifar-1.pth')
torch.save(conv_net, 'rot-trained-conv-cifar-1.pth')
torch.save(convu_net, 'upright-trained-conv-cifar-1.pth')